<center><H1>DataRobot AutoML end-to-end with Amazon Athena</H1></center>

<table border="0" cellspacing="0" cellpadding="0">
<td><img src="https://www.datarobot.com/wp-content/uploads/2021/08/DataRobot-logo-color.svg" height=200px width=200px>
</td>
<td><font size=10> + </font> </td>
<td> <img src="https://vectorwiki.com/images/1BalA__aws-athena.svg" height=100px width=100px> </td>

Author: Biju Krishnan

[API reference documentation](https://docs.datarobot.com/en/docs/api/reference/index.html)

<font>
This example notebook outlines the following tasks: <p>
<ol>
<li> Read in an Amazon Athena table and upload it to DataRobot's AI Catalog </li>
<li> Create a project with the dataset</li>
<li> Deploy the top performing model to a DataRobot prediction server </li>
<li> Make batch predictions with a test dataset </li>
</ol>
<p>
</font>

## Setup

### Import libraries

In [ ]:
import datarobot as dr

### Bind variables

In [ ]:
# These variables can aso be fetched from a secret store or config files
DATAROBOT_ENDPOINT = "https://app.eu.datarobot.com/api/v2"
# The URL may vary depending on your hosting preference, the above example is for DataRobot EU Managed AI Cloud

DATAROBOT_API_TOKEN = "<INSERT YOUR DataRobot API Token>"
# The API Token can be found by click the avatar icon and then </> Developer Tools

client = dr.Client(
    token=DATAROBOT_API_TOKEN,
    endpoint=DATAROBOT_ENDPOINT,
    user_agent_suffix="AIA-E2E-AWS-16",  # Optional but helps DataRobot improve this workflow
)

dr.client._global_client = client

AWS_KEY = "<INSERT YOUR AWS ACCESS KEY>"  # Enter your AWS Key ID
AWS_SECRET = "<INSERT YOUR AWS SECRETS>"  # Enter your AWS Secret

### Connect to DataRobot

You can read more about different options for [connecting to DataRobot from the client](https://docs.datarobot.com/en/docs/api/api-quickstart/api-qs.html).

In [ ]:
dr.Client(token=DATAROBOT_API_TOKEN, endpoint=DATAROBOT_ENDPOINT, ssl_verify=False)

In [ ]:
# This line reads the driver object needed for creating a datastore
athena_driver = [drv for drv in dr.DataDriver.list() if drv.canonical_name == "AWS Athena (v5)"][-1]

## Import data

### Create a data connection

Use the cell below to define the parameters required to make a connector.

In [ ]:
athena_s3_bucket = "e2eaccelerator09122022"  # Specifythe name of the bucket followed by any prefix, later you format it as an S3 URI

jdbc_url = "jdbc:awsathena://athena.eu-west-1.amazonaws.com;AwsRegion=eu-west-1;S3OutputLocation=s3://{}/".format(
    athena_s3_bucket
)

query = 'SELECT * FROM "new_york_taxi"."input" limit 10000;'

In [ ]:
# Create a data connection (AKA a datastore)

DR_DATASTORE_NAME = "ATHENA Data Connection"  # This name can be altered

# Checking if datastore already exists
for dstore in dr.DataStore.list():
    if dstore.canonical_name == DR_DATASTORE_NAME:
        datastore_flag = False
        datastore = dstore
        break
    else:
        datastore_flag = True

if datastore_flag:
    datastore = dr.DataStore.create(
        data_store_type="jdbc",
        canonical_name="ATHENA Data Connection",  # This name can be replaced
        driver_id=athena_driver.id,
        jdbc_url=jdbc_url,
    )

In [ ]:
# Create a data connection based on a query
# The Athena JDBC driver only supports query-based ingestion

params = dr.DataSourceParameters(data_store_id=datastore.id, query=query)

DR_DATASOURCE_NAME = "ATHENA Data Source"  # This name can be altered

for dsource in dr.DataSource.list():
    if dsource.canonical_name == DR_DATASOURCE_NAME:
        datasource_flag = False
        datasource = dsource
        break
    else:
        datasource_flag = True

if datasource_flag:
    datasource = dr.DataSource.create(
        data_source_type="jdbc",
        canonical_name="ATHENA Data Source",  # This name can be altered
        params=params,
    )

In [ ]:
# This code snippet creates a snapshot of the Athena table and stores it in the AI Catalog

datarobot_dataset = dr.Dataset.create_from_data_source(
    data_source_id=datasource.id, username=AWS_KEY, password=AWS_SECRET
)

## Create a project and initiate Autopilot

In [ ]:
# This cell will take several minutes to complete execution
# An AutoML project named "E2E Demo Amazon Athena" is created with "tip_amount" as the target column
# Quick mode is designated, however other modes are also available


EXISTING_PROJECT_ID = None  # If you've already created a project, replace None with the ID here

if EXISTING_PROJECT_ID is None:
    # Create project and pass in data
    project = dr.Project.create_from_dataset(
        datarobot_dataset.id, project_name="E2E Demo Amazon Athena"
    )

    # Set the project target to the appropriate feature. Use the LogLoss metric to measure performance
    project.analyze_and_model(target="tip_amount", mode=dr.AUTOPILOT_MODE.QUICK, worker_count="-1")
else:
    # Fetch the existing project
    project = dr.Project.get(EXISTING_PROJECT_ID)

project.wait_for_autopilot(check_interval=30)

### Get the top-performing model

Once the AutoML project is complete, select the top-performing model on the Leaderboard based on the chosen metric for deployment.

In [ ]:
def sorted_by_metric(models, test_set, metric):
    models_with_score = [model for model in models if model.metrics[metric][test_set] is not None]

    return sorted(models_with_score, key=lambda model: model.metrics[metric][test_set])


models = project.get_models()

metric = project.metric

# Get the top-performing model
model_top = sorted_by_metric(models, "crossValidation", metric)[0]

print(
    """The top performing model is {model} using metric, {metric}""".format(
        model=str(model_top), metric=metric
    )
)

### Deploy a model

Note that steps in the following sections require DataRobot MLOps licensed features. Contact your DataRobot account representatives if you are missing some licensed MLOps features.

In [ ]:
# Get the prediction server
prediction_server = dr.PredictionServer.list()[0]

# Create a deployment
deployment = dr.Deployment.create_from_learning_model(
    model_top.id,
    label="E2E Amazon Athena Test",
    description="Model trained on New York Taxi trips dataset",
    default_prediction_server_id=prediction_server.id,
)
deployment.id

### Make predictions

DataRobot's batch predictions API is capable of directly reading and writing to Amazon S3 storage. 

In [ ]:
# To run a batch prediction job you need to store the Amazon Athena Credentials in the DataRobot credentials manager

DR_CREDENTIAL_NAME = "Amazon Athena Credentials"  # Choose a name
for cred in dr.Credential.list():
    if cred.name == DR_CREDENTIAL_NAME:
        cred_flag = False
        athena_credential_id = cred.credential_id
        break
    else:
        cred_flag = True

# Create credentials in DataRobot credential store if they do not exist
if cred_flag:
    credential = dr.Credential.create_basic(
        name=DR_CREDENTIAL_NAME,  # The username and password is the AWS KEY and SECRET respectively
        user=AWS_KEY,
        password=AWS_SECRET,
    )
    athena_credential_id = credential.credential_id

print(athena_credential_id)

In [ ]:
DR_CREDENTIAL_NAME = "AWS S3 Credentials"  # Choose a name as per your convenience
for cred in dr.Credential.list():
    if cred.name == DR_CREDENTIAL_NAME:
        cred_flag = False
        s3_credential_id = cred.credential_id
        break
    else:
        cred_flag = True

# Create credentials in DataRobot credential store if it does not exist
if cred_flag:
    credential = dr.Credential.create_s3(
        name=DR_CREDENTIAL_NAME,
        aws_access_key_id=AWS_KEY,
        aws_secret_access_key=AWS_SECRET,
        # aws_session_token= <Optional>
    )
    s3_credential_id = credential.credential_id

print(s3_credential_id)

### Batch predictions snippet

The snippet below provides sample code to demonstratehow to make batch predictions to and from Amazon S3.

In [ ]:
# This example scores the training data but there needs to be an Athena table with test data.

job = dr.BatchPredictionJob.score(
    deployment=deployment.id,
    intake_settings={
        "type": "jdbc",
        "query": "select * from new_york_taxi.input limit 1000",  # This has to be a query, since the JDBC driver does not seem to understand table schema structure
        "data_store_id": datastore.id,  # The ID of the datastore you want
        "credential_id": athena_credential_id,  # The credentialid of the credentials stored in your credentials manager
    },
    output_settings={
        "type": "s3",
        "url": "s3://e2eaccelerator09122022/predictions/output/new_york_taxi_predictions.csv",  # Note this has to be a filename and not just a bucket name
        "credential_id": s3_credential_id,
    },
)
job.wait_for_completion()
job.get_status()

<font family=verdana>
<p>
The output of the batch predictions is available under the path s3://e2eaccelerator09122022/predictions/output/
<pre><code><font color=grey size=1>
aws s3 ls s3://e2eaccelerator09122022/predictions/output/new_york_taxi_predictions.csv
2022-12-12 17:47:49      22725 new_york_taxi_predictions.csv
</font></code></pre>
</font>
